# Data Preprocessing

1. boxoffice
2. detail
3. rating
4. data table
5. search result  
> _search result was done in Crawler.ipynb_

In [2]:
import pandas as pd
import datetime
import requests
import json
import csv
# import matplotlib as plt

#-*- coding: utf-8 -*-

# kim key no.1 : c15fcbaab76cc525e9342c5e4dda712f
# kim key no.2 : c3268b0918814ae0a2bc298f7ba893a3
# lee key no.1 : 77a87d41d6e2cbe3131a190d5c48514e
# lee key no.2 : e53650978fc97b47c7f932136ce2c08a

## Crawling movie boxoffice data



In [117]:
key1 = 'e53650978fc97b47c7f932136ce2c08a'

def crawl_movie_boxoffice_data(start_date, end_date, key = key1) :
    result_list = []
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json'
    
    for single_date in pd.date_range(start_date, end_date) :
        for multi, rep in zip(['Y', 'Y', 'N', 'N'], ['K', 'F', 'K', 'F']) :
            payload = {
                'key' : key,
                'targetDt' : single_date.strftime('%Y%m%d'),
                'itemPerPage' : '10',
                'multiMovieYn' : multi,
                'repNationCd' : rep
            }
            
            req = requests.get(url, params = payload)

            for item in req.json()['boxOfficeResult']['dailyBoxOfficeList'] :
                temp_list = []
                header_list = []
                for header, value in item.items() :
                    header_list.append(header)
                    temp_list.append(value)
                    
                temp_list.append(single_date)
                header_list.append('CurrentDate')
                
                temp_list.append(multi)
                header_list.append('multi')
                
                temp_list.append(rep)
                header_list.append('Nation')
                
                result_list.append(temp_list)
                
    return pd.DataFrame(result_list, columns = header_list)

movie_boxoffice_list = crawl_movie_boxoffice_data('20020101', '20021231', key = key1)
movie_boxoffice_list.to_csv('movie_boxoffice_2002.csv', index = False)

UnboundLocalError: local variable 'header_list' referenced before assignment

In [25]:
## concatenate boxoffice files
# 2013 ~ 2018

all_filenames = []
for i in range(2013, 2019) :
    all_filenames.append('movie_boxoffice_'+str(i)+'.csv')
movie_boxoffice_combined = pd.concat([pd.read_csv(f) for f in all_filenames ])

movie_boxoffice_combined.to_csv('movie_boxoffice.csv', index = False, encoding = 'utf-8')

In [19]:
## test

movie_boxoffice_file_2017 = open('movie_boxoffice_2017.csv', 'r', encoding = 'utf-8')
movie_boxoffice_file_2018 = open('movie_boxoffice_2018.csv', 'r', encoding = 'utf-8')

fa=pd.read_csv(movie_boxoffice_file_2017)
fb=pd.read_csv(movie_boxoffice_file_2018)

print(len(fa))
print(len(fb))
with open('movie_boxoffice.csv', 'r', encoding = 'utf-8') as infile :
    fc=pd.read_csv(infile)
    print(len(fc))

14446
14560
29006


In [26]:
movie_boxoffice_list = pd.read_csv('movie_boxoffice.csv')

len(movie_boxoffice_list)

86747

## Crawling movie rating

from Naver movie

In [27]:
# movie_rating_list = pd.DataFrame({'movieCd' : movie_boxoffice_f.movieCd.drop_duplicates(), 'rating' : 0})
movie_rating_list = pd.DataFrame({'movieCd' : movie_boxoffice_list.movieCd.drop_duplicates(), 'rating' : 0})

len(movie_rating_list)

6195

In [28]:
movie_rating_list.to_csv('movie_rating_list.csv', index = False, encoding = 'utf-8')

In [45]:
## sorting movie_rating_list
# and divide 3 parts to use seperate keys

movie_rating_list=movie_rating_list.sort_values(by='movieCd', ascending=True).reset_index()
mrl1 = movie_rating_list.iloc[0:3000]
mrl2 = movie_rating_list.iloc[3000:6000]
mrl3 = movie_rating_list.iloc[6000:]



3000
3000
195


In [50]:
key2 = '77a87d41d6e2cbe3131a190d5c48514e'

def crawl_movie_detail_data(lists, key = key2) :
    movie_detail_json = dict()
    url = 'http://www.kobis.or.kr/kobisopenapi/webservice/rest/movie/searchMovieInfo.json'
    
    for movieCd in lists['movieCd'] :
        payload = {
            'key' : key,
            'movieCd' : movieCd
        }
        req = requests.get(url, params = payload)
        
        try :
            movie_detail_json[movieCd] = req.json()['movieInfoResult']['movieInfo']
        except :
            print("Error !")
            print(req.json())
        
    return movie_detail_json

movie_detail = crawl_movie_detail_data(mrl3)

with open('movie_detail_3.json', 'w', encoding = 'utf-8') as output :
    json.dump(movie_detail, output)

In [11]:
with open('movie_detail_1.json', 'r') as f :
    movie_detail_1 = json.load(f)
with open('movie_detail_2.json', 'r') as f :
    movie_detail_2 = json.load(f)
with open('movie_detail_3.json', 'r') as f :
    movie_detail_3 = json.load(f)

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [30]:
## concatenate movie detail(json type) files
# 2013 ~ 2018

movie_detail = dict()
for j in movie_detail_1 :
    movie_detail[j] = movie_detail_1[j]

for j in movie_detail_2 :
    movie_detail[j] = movie_detail_2[j]

for j in movie_detail_3 :
    movie_detail[j] = movie_detail_3[j]

with open('movie_detail.json', 'w', encoding = 'utf-8') as output :
    json.dump(movie_detail, output)

3000
6000
6195


In [51]:
movie_rating_list = pd.read_csv('movie_rating_list.csv')

movie_rating_list = movie_rating_list.sort_values(by='movieCd', ascending = True)

movie_rating_list = movie_rating_list.drop(columns=['index'])

movie_rating_list.to_csv('movie_rating_list.csv', index = False, encoding = 'utf-8')

In [57]:
movie_rating_list = pd.read_csv('movie_rating_list.csv')

movie_detail[movie_rating_list.iloc[i,0]]['movieNm']

'오렌지필름 기획전 - do not reply'

In [247]:
## Crawling user's movie rating
# from Naver api (Naver movie)

# with open('movie_detail.json', 'r', encoding = 'utf-8') as f :
#     movie_detail = json.load(f)
# movie_rating_list = pd.read_csv('movie_rating_list.csv')

for i in range(len(movie_rating_list)) :
    url = 'https://openapi.naver.com/v1/search/movie.json'
    
    mv = movie_detail[movie_rating_list.iloc[i, 0]]
    payload = {
        'query' : mv['movieNm']
        , 'display' : 10
    }
    
    headers = {
        'X-Naver-Client-Id' : 'V8LONuGkI0U72r0RY3yo',
        'X-Naver-Client-Secret' : 'VKW2Diwq5d'
    }
    
    if (movie_rating_list.iloc[i, 1] == 0) | (movie_rating_list.iloc[i, 1] == 'NA') :
        # print(movie_rating_list.iloc[i, 0], mv['movieNm'] + mv['movieNmEn'])
        try :
            req = requests.get(url, headers = headers, params = payload)
            req.json()['total']
        except :
            movie_rating_list.iloc[i, 1] = 'NA'
        else :
            if(req.json()['total'] == 1) :
                movie_rating_list.iloc[i, 1] = req.json()['items'][0]['userRating']
            else :
                d = movie_detail[movie_rating_list.iloc[i, 0]]['directors']
                flag = False
                for jj in d :
                    for j in req.json()['items'] :
                        if(j['director'][:-1] == jj['peopleNm']) :
                            movie_rating_list.iloc[i,1]=j['userRating']
                            flag = True
                        if(flag == True) :
                            break
                    if(flag == True) :
                        break

[{'title': '<b>솔로몬 왕의 보고</b>', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=13074', 'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/0130/A3074-01.jpg', 'subtitle': "King Solomon's Mines", 'pubDate': '1950', 'director': '콤프톤 베넷|앤드류 마튼|', 'actor': '데보라 카|스튜어트 그레인저|리차드 칼슨|휴고 하스|로웰 길모어|키무르시|시리아크|세카리온고|바지가|', 'userRating': '6.83'}, {'title': '<b>솔로몬</b> 왕의 광산', 'link': 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=15671', 'image': 'https://ssl.pstatic.net/imgmovie/mdi/mit110/0156/15671_P01_125747.jpg', 'subtitle': "King Solomon's Mines", 'pubDate': '1937', 'director': '로버트 스티븐슨|', 'actor': '폴 로브슨|세드릭 하드윅|', 'userRating': '0.00'}]


In [64]:
movie_rating_list.to_csv('movie_rating_list.csv', index = False, encoding = 'utf-8')

# movie_data

## movie_detail.json

- movieCd  
- movieNm  
- movieNmEn  
- movieNmOg ''  
- showTm '144'  
- prdtYear '2016'  
- openDt '20160601'  
- prdtStatNm '개봉'  
- typeNm  
- nations  
    * nationNm  
- genres  
    * genreNm  
- diretors  
    * peopleNm  
    * peopleNmEn  
- actors  
    * peopleNm  
    * peopleNmEn  
    * cast  
    * castEn  
- showTypes  
    * showTypeGroupNm  
    * showTypeNm  
- companys  
    * companyCd  
    * companyNm  
    * companyEn  
    * companyPartNm '제작사'  
- audits  
    * auditNo '2016-MF00708'  
    * watchGradeNm '청소년관람불가'  
- staffs  
    * peopleNm  
    * peopleNmEn  
    * staffRoleNm  
  
## movie_boxoffice.csv  
- rnum  
- rank  
- rankInten  
- rankOldAndNew   
- movieCd   
- movieNm  
- openDt  
- salesAmt  
- salesShare  
- salesInten  
- salesChange  
- salesAcc  
- audiCnt  
- audiInten  
- audiChange  
- audiAcc  
- scrnCnt  
- showCnt 
- CurrentDate
- multi  
- Nation  

In [3]:
movie_boxoffice = pd.read_csv('movie_boxoffice.csv')

movie_group = movie_boxoffice.groupby(['movieCd'])
movie_data = pd.DataFrame({
                        'movieNm' : movie_group['movieNm'].max(),
                        'salesAcc' : movie_group['salesAcc'].max(),
                        'salesAvg' : movie_group['salesAmt'].mean(),
                        'screen' : movie_group['scrnCnt'].sum(),
                        'show' : movie_group['showCnt'].sum(),
                        'audienceAcc' : movie_group['audiAcc'].max(),
                        'audienceAvg' : movie_group['audiCnt'].mean(),
                        'multi' : movie_group['multi'].max(),
                        'nation' : movie_group['Nation'].max(),
                        'openDate' : movie_group['openDt'].min()
                        }).reset_index()

top10_sales = movie_data.sort_values(by = 'salesAcc', ascending = False).head(10).reset_index()
top10_audiences = movie_data.sort_values(by = 'audienceAcc', ascending = False).head(10).reset_index()

In [101]:
top10_sales

,index,movieCd,movieNm,salesAcc,salesAvg,screen,show,audienceAcc,audienceAvg,multi,nation,openDate
0,1458,20129370,명량,135756027310,1.190767e+09,40563,188579,17615437,154508.368421,N,K,2014-07-30
1,836,20090834,아바타,125304346000,2.672213e+07,619,1931,13338863,2380.333333,Y,F,2009-12-17
2,3269,20150976,신과함께-죄와 벌,115705891137,1.428419e+09,55203,213194,14411477,177911.975309,N,K,2017-12-20
3,2117,20137048,국제시장,110938092730,6.338978e+08,52226,212606,14262922,81495.977143,N,K,2014-12-17
4,2874,20148048,베테랑,105168923250,1.001502e+09,43995,199090,13414136,127735.523810,N,K,2015-08-05
5,6040,20186202,신과함께-인과 연,102664946909,1.425758e+09,44953,179889,12274846,170464.736111,N,K,2018-08-01
6,5394,20177478,어벤져스: 인피니티 워,99915606269,1.331828e+09,56248,240284,11211428,149440.826667,N,F,2018-04-25
7,3006,20148851,암살,98467187781,9.376947e+08,42924,175121,12706663,120997.695238,N,K,2015-07-22
8,4213,20162869,택시운전사,95868560649,9.129971e+08,46176,184080,12189141,116080.419048,N,K,2017-08-02
9,1263,20124034,도둑들,93667072500,6.140000e+05,2,5,12983821,154.000000,N,K,2012-07-25


In [103]:
top10_audiences

,index,movieCd,movieNm,salesAcc,salesAvg,screen,show,audienceAcc,audienceAvg,multi,nation,openDate
0,1458,20129370,명량,135756027310,1.190767e+09,40563,188579,17615437,154508.368421,N,K,2014-07-30
1,3269,20150976,신과함께-죄와 벌,115705891137,1.428419e+09,55203,213194,14411477,177911.975309,N,K,2017-12-20
2,2117,20137048,국제시장,110938092730,6.338978e+08,52226,212606,14262922,81495.977143,N,K,2014-12-17
3,2874,20148048,베테랑,105168923250,1.001502e+09,43995,199090,13414136,127735.523810,N,K,2015-08-05
4,836,20090834,아바타,125304346000,2.672213e+07,619,1931,13338863,2380.333333,Y,F,2009-12-17
5,1263,20124034,도둑들,93667072500,6.140000e+05,2,5,12983821,154.000000,N,K,2012-07-25
6,1402,20127593,7번방의 선물,91433282670,7.199427e+08,37820,166795,12811435,100876.598425,N,K,2013-01-23
7,3006,20148851,암살,98467187781,9.376947e+08,42924,175121,12706663,120997.695238,N,K,2015-07-22
8,1280,20124079,"광해, 왕이 된 남자",88908845769,5.840625e+05,16,36,12323510,256.937500,N,K,2012-09-13
9,6040,20186202,신과함께-인과 연,102664946909,1.425758e+09,44953,179889,12274846,170464.736111,N,K,2018-08-01


In [105]:
tmp_movie_data = movie_data

In [113]:
tmp_movie_data['salesPerAudit'] = movie_data['salesAcc']/movie_data['audienceAcc']

top10_salesPerAudit = tmp_movie_data.sort_values(by = 'salesPerAudit', ascending = False).head(10).reset_index()

NameError: name 'movie_data_table' is not defined

In [115]:
tmp_movie_data.head(5)

,movieCd,movieNm,salesAcc,salesAvg,screen,show,audienceAcc,audienceAvg,multi,nation,openDate,salesPerAudit
0,19278002,제너럴,857000,2.680000e+05,1,1,177,40.00,N,F,,4841.807910
1,19288001,잔다르크의 수난,2192000,7.306667e+05,3,18,1086,362.00,N,F,,2018.416206
2,19308006,모로코,3535000,5.387500e+05,4,19,1642,268.75,N,F,,2152.862363
3,19328008,스카페이스,4104000,1.026000e+06,4,16,2032,508.00,Y,F,,2019.685039
4,19338008,뮤지컬 42번가,5201000,7.556000e+05,5,25,2579,374.80,Y,F,,2016.673129


In [118]:
movie_detail['20144444']

{'movieCd': '20144444',
 'movieNm': '아가씨',
 'movieNmEn': 'The Handmaiden',
 'movieNmOg': '',
 'showTm': '144',
 'prdtYear': '2016',
 'openDt': '20160601',
 'prdtStatNm': '개봉',
 'typeNm': '장편',
 'nations': [{'nationNm': '한국'}],
 'genres': [{'genreNm': '스릴러'}, {'genreNm': '드라마'}],
 'directors': [{'peopleNm': '박찬욱', 'peopleNmEn': 'PARK Chan-wook'}],
 'actors': [{'peopleNm': '김민희',
   'peopleNmEn': 'KIM Min-hee',
   'cast': '아가씨',
   'castEn': ''},
  {'peopleNm': '김태리', 'peopleNmEn': 'KIM Tae-ri', 'cast': '하녀', 'castEn': ''},
  {'peopleNm': '하정우', 'peopleNmEn': 'HA Jung-woo', 'cast': '백작', 'castEn': ''},
  {'peopleNm': '조진웅',
   'peopleNmEn': 'CHO Jin-woong',
   'cast': '이모부',
   'castEn': ''},
  {'peopleNm': '김해숙',
   'peopleNmEn': 'KIM Hae-sook',
   'cast': '여집사',
   'castEn': ''},
  {'peopleNm': '문소리', 'peopleNmEn': 'MOON So-ri', 'cast': '이모', 'castEn': ''},
  {'peopleNm': '최병모',
   'peopleNmEn': 'CHOE Byeong-mo',
   'cast': '손님',
   'castEn': ''},
  {'peopleNm': '김인우', 'peopleNmEn': 'K

In [264]:
movie_data[movie_data['movieCd']=='19278002']['multi'].iloc[0]

'N'

In [269]:
## movie_data_key / only multi = 'Y'
# actors
# directors
# genreNm
# nationNm
# showTypeNm
# typeNm
# watchGradeNm

with open('movie_detail.json', 'r', encoding = 'utf-8') as mdf :
    movie_detail = json.load(mdf)



def construct_movie_key(ref_md) :
    a_key = 0
    d_key = 0
    g_key = 0
    n_key = 0
    s_key = 0
    t_key = 0
    w_key = 0
    actor_list = []
    director_list = []
    genre_list = []
    nation_list = []
    showtype_list = []
    type_list = []
    grade_list = []
    
    result_list = []
    header_list = ['movieCd', 'actorNm', 'genreNm', 'nationNm', 'showTypeNm', 'typeNm', 'watchGradeNm']
    # print(ref_md['20144444'])
    for i in ref_md :
        # print(i)
        ## actor
        mm = movie_data[movie_data['movieCd'] == i]
        if(mm['multi'].iloc[0] == 'Y') & (mm['nation'].iloc[0] == 'K') :
            try :
                for j in ref_md[i]['actors'] :
                    if j['peopleNm'] not in actor_list :
                        print(i + j['peopleNm'])
                        actor_list.append(j['peopleNm'])
                    # print(j)
            except :
                print("Error ! | actor")

            ## director
            try :
                for j in ref_md[i]['directors'] :
                    if j['peopleNm'] not in director_list :
                        director_list.append(j['peopleNm'])
                    # print(j)
            except :
                print("Error ! | director")

            ## genre
            try :
                for j in ref_md[i]['genres'] :
                    if j['genreNm'] not in genre_list :
                        genre_list.append(j['genreNm'])
                    # print(j)
            except :
                print("Error ! | genre")

            ## nation
            try :
                for j in ref_md[i]['nations'] :
                    if j['nationNm'] not in nation_list :
                        nation_list.append(j['nationNm'])
                    # print(j)
            except :
                print("Error ! | nation")

            ## showtype
            for j in ref_md[i]['showTypes'] :
                if j['showTypeGroupNm'] not in showtype_list :
                    showtype_list.append(j['showTypeGroupNm'])
                # print(j)

            ## type
            try :
                # print(ref_md[i]['typeNm'])
                if ref_md[i]['typeNm'] not in type_list :
                    type_list.append(ref_md[i]['typeNm'])
            except :
                print("Error ! | type")


            ## grade
            try :
                for j in ref_md[i]['audits'] :
                    if j['watchGradeNm'] not in grade_list :
                        grade_list.append(j['watchGradeNm'])
                    # print(j)
            except :
                print("Error ! | grade")
        
        
    # return pd.DataFrame(result_list, columns = header_list)
    return (actor_list, director_list, genre_list, nation_list, showtype_list, type_list, grade_list)

movie_data_key = construct_movie_key(movie_detail)

19580020최은희
19580020이민자
19580038이경태
19580038이대엽
19580038도금봉
19580073이룡
19580073장민호
19600003김진규
19600003김혜정
19600003황정순
19600003엄앵란
19600003윤정란
19600017신성일
19600017주증녀
19600018이은심
19600018안성기
19618005문정숙
19618014김지미
19618028김승호
19618028황해
19618028허장강
19638008구봉서
19638008최성
19638008최무룡
19640025윤일봉
19640025이예춘
19650018김희갑
19650018박노식
19668013남궁원
19688044신영균
19688044최남현
19688044이풍구
19688097홍세미
19688097김정옥
19698042김정훈
19698042김지영
19698042문희
19708104김성옥
19708104김동원
19718198김희준
19720151김희라
19720151장동휘
19720151이순재
19728178리칭
19738119오수미
19748089이대근
19760026이승룡
19760026진추하
19770051송재호
19770051염복순
19770051최불암
19770055장미희
19770055김추련
19770055신광일
19770058이신재
19770058이경희
19770058송승환
19788022이영수
19788022조윤숙
19788022김신재
19789099혜은이
19789099김현
19789099이미경
19789099백일섭
19789099홍윤정
19790024유지인
19790024남능미
19790038이영하
19790038정영숙
19790038허진
19790055유장현
19790055정애란
19798135김석훈
19800015하명중
19800015정애리
19800015김미숙
19800015이문희
19800015한국남
19810007김영애
19810007원미경
19810007이혜숙
19810007정한용
19810007한진희
19810007박원숙

20101244정아영
20101244강석
20101244강영수
20101244윤보경
20101244신혜옥
20101244박세희
20101244박진아
20101245심이영
20101245서옥별
20101245임종윤
20101245정재웅
20101245이정은
20101245유하준
20101245강민영
20101245송진열
20101245김태훈
20101245박미현
20101247이경준
20101247김민혁
20101247양정원
20101247성민철
20101247조은
20101247박철용
20101249소준문
20101249정욜
20101249최영수
20101249장병권
20101339남무성
20101339조성준
20101339김희동
20101339최정훈
20101339홍종민
20101339김은민
20101339박종우
20101339조윤성
20101339황태연
20101339노상현
20101339이동기
20101339차정훈
20101339유영수
20101339임인건
20101339임재필
20101339김수열
20101339전계숙
20101339이원술
20101339이한진
20101339김준
20101343박해일
20101343이민지
20101343박지윤
20101343탁도연
20101343차주동
20101343김택규
20101343이경원
20101343박세종
20101343이민아
20101344박정민
20101344이초희
20101344이재연
20101344배제기
20101392박신혜
20101392오연서
20101392송창의
20101392신정희
20101392박진희
20101392조효민
20101392이태연
20110254김상중
20110254송선미
20110254김의성
20110254백종학
20110254정지형
20110254배유람
20110254안재홍
20110985이바울
20110985염현준
20110985김새벽
20110985임형국
20110985정건영
20110985서민경
20110985최장원
20110985오희준
20111021곽규석
20111031

20128609갤럭시 익스프레스
20128609백승화
20129089이동욱
20129099박주영
20129211조은아
20129211조용석
20129211이석
20129211강석철
20129332최경준
20129332강연정
20129332박지성
20129332정보창
20129332신재혁
20129786장영남
20129786마동석
20129786황태광
20129786엄태구
20129786김형종
20129786추귀정
20129786홍기준
20129786도용구
20129786강민선
20129786김시내
20129786홍석재
20129786김정숙
20129786류성진
20129786문예진
20129786송민승
20129786이병삼
20130244정영진
20130244김홍파
20130244박형우
20130244박재현
20130244임호경
20130244장문진
20130244정보경
20130244이상록
20130244이소영
20130244홍승주
20130244김경화
20130244이은혜
20130244김남숙
20130244황치연
20130260조은지
20130260박재철
20130260강민지
20130260한수용
20130260최혁
20130481정겨운
20130481윤승아
20130481임현성
20130481가심현
20130481신수연
20130481오혜원
20130481우문기
20130481윤찬영
20130481이보경
20130481이지안
20130481장철민
20130481정유영
20130481채수아
20130481한재웅
20130481형슬우
20130481홍상원
20130481김경수
20130481김영웅
20130481박소담
20130481문서연
20130482권율
20130482배범식
20130482성스러운
20130482송아미
20130482오수빈
20130482유영선
20130482유원상
20130482이규봉
20130482이영상
20130482이우람
20130482이자연
20130482김승곤
20130482정수련
20130482박헌수
20130482황선영


20141610신효진
20141610박정훈
20141610강보경
20141611정하담
20141611오창경
20141611백현주
20141611최영록
20141611이준수
20141611최수안
20141611조성호
20141612백수장
20141613전범수
20141613이준민
20141613정넘쳐
20141613한을희
20141613이상미
20141613이수연
20141613이용표
20141613김종태
20141614차예련
20141614이세야 유스케
20141702강명자
20141702김진숙
20141702권수정
20141702신순애
20141702이성옥
20141702장영선
20141704신창열
20141704김호경
20141704유진희
20141705김조광수
20141705한가람
20141705김선경
20141705김용균
20141705윤소라
20141705이준호
20141706이상호
20141706이종인
20141708정승환
20141708한민수
20141708유만균
20141708김정호
20141708장종호
20141708정영훈
20141746문채원
20141746고수
20142221문정현
20142221블라디미르 토도로비치
20142221다니엘 루디 하리얀토
20142401팽지인
20142401서양선
20142401유나론
20142403박민상
20142403박상면
20142403춘자
20142403김형범
20142624조민수
20142624진성민
20142624허범욱
20142624이성일
20142624이규창
20142661전주연
20142661김진주
20142661민병애
20142661황보민경
20142661정혜선
20142661전원주
20142661김명국
20142690김갑수
20142706이동우
20142706임재신
20142706김윤태
20142798권오중
20142798옥한흠
20143081강석우
20143081최강희
20143081이광기
20143252정준교
20143252이하민
20143252강필선
20143382추민주
20143382

20155483배남식
20155483김순덕
20155483임간난
20155483권민기
20155483강해춘
20155923이원종
20155923이장숙
20155923정성윤
20155923박준하
20155923서대현
20155923최지명
20155923신예진
20155923김병오
20155923남기웅
20155926정단우
20155926엘리자베스 가르시아
20155926박하나
20155926이태경
20155926김영
20156047최경덕
20156188양지원
20156188여정민
20156188유영제
20156188하지현
20156281이해영
20156281전홍석
20156281정혜지
20156281최경옥
20156281최성현
20156281안소영
20156281윤희경
20156281손범수
20156353다나
20156353이설애
20156353장석우
20156353한성수
20156498김승헌
20156498김찬형
20156498김철민
20156498박지현
20156498왕진성
20156498정수환
20156498채종규
20157024김민체
20157024권중목
20157024이민서
20157024전다솔
20157024박진국
20157024정현우
20157024구준희
20157024차승기
20157024이종선
20157024염문경
20157024전희련
20157024권문선
20157024권정수
20157024장해숙
20157024송성해
20157024최종문
20157024이재훤
20157024승의열
20157024윤광현
20157024이문주
20157064고보결
20157064신문성
20157064김명성
20157064박선주
20157064방한솔
20157064최승호
20157191권민중
20157191최할리
20157191신소미
20157191경준
20157191주연서
20157191나동주
20157191류제현
20157191문진영
20157191안정균
20157191이태헌
20157191임창용
20157191조솔
20157431김한민
20157431오타니 료

20172625박찬수
20172625오유정
20172625이아린
20172625이연화
20172625이윤서
20172625임경빈
20172625정길종
20172625정종화
20172626박중하
20172626신지현
20172641마리안느 스퇴거
20172641마가렛 피사렉
20172642성병숙
20172642윤상화
20172642김래원
20172863박산다라
20172863한재석
20172863박혜경
20172863신민정
20172863정용환
20172863이정우
20172863노현석
20173235진현
20173301윤안나
20173301안은새
20173301강다연
20173301도세진
20173301민주홍
20173301박사라
20173301이상운
20173301진미혜
20173301최용수
20173612박성웅
20173612박진성
20173612서지혜
20173612조윤담
20173612조혜림
20173612이장호
20173612이혜수
20173732김상범
20173732김상우
20173732김시영
20173732박승찬
20173732박영진
20173732배준성
20173732서영민
20173732송승현
20173732이경진
20173732이민재
20173732이범희
20173732이순옥
20173732이종재
20173732인영교
20173732장승화
20173732주소윤
20173732차재민
20173732하유정
20173734선우정아
20173734김민아
20173734김유민
20173734김윤애
20173734김은정
20173734김종훈
20173734김지호
20173734김한솔
20173734남승우
20173734문광주
20173734백성빈
20173734서지영
20173734송지연
20173734유정호
20173734윤진
20173734이경은
20173734이슬기
20173734이슬아
20173734이영호
20173734이원규
20173734이진아
20173734이호준
20173734정선희
20173734정순태
20173734정요한
2017373

In [270]:
len(movie_data_key)

7

In [271]:
md_key_list = ['actor', 'director', 'genre', 'nation', 'showType', 'type', 'grade']
movie_data_table = dict()
for j in range(0,7) :
    cnt = 0
    md_dict = dict()
    for i in movie_data_key[j] :
        md_dict[i] = json.dumps(int(cnt))
        cnt = cnt + 1
    movie_data_table[md_key_list[j]] = md_dict

In [281]:
len(movie_data_table['director'])

4166

In [273]:
with open('movie_data_table_nonmulti_kr.json', 'w') as output :
    json.dump(movie_data_table, output)

### movie_data_table.csv

In [276]:
## movie_data_key
# actors
# directors
# genreNm
# nationNm
# showTypeNm
# typeNm
# watchGradeNm

with open('movie_detail.json', 'r', encoding = 'utf-8') as mdf :
    movie_detail = json.load(mdf)



def construct_movie_key(ref_md) :
    a_key = 0
    d_key = 0
    g_key = 0
    n_key = 0
    s_key = 0
    t_key = 0
    w_key = 0
    actor_list = []
    director_list = []
    genre_list = []
    nation_list = []
    showtype_list = []
    type_list = []
    grade_list = []
    
    result_list = []
    header_list = ['movieCd', 'actorNm', 'genreNm', 'nationNm', 'showTypeNm', 'typeNm', 'watchGradeNm']
    # print(ref_md['20144444'])
    for i in ref_md :
        # print(i)
        ## actor
        try :
            for j in ref_md[i]['actors'] :
                if j['peopleNm'] not in actor_list :
                    print(i + j['peopleNm'])
                    actor_list.append(j['peopleNm'])
                # print(j)
        except :
            print("Error ! | actor")
        
        ## director
        try :
            for j in ref_md[i]['directors'] :
                if j['peopleNm'] not in director_list :
                    director_list.append(j['peopleNm'])
                # print(j)
        except :
            print("Error ! | director")
            
        ## genre
        try :
            for j in ref_md[i]['genres'] :
                if j['genreNm'] not in genre_list :
                    genre_list.append(j['genreNm'])
                # print(j)
        except :
            print("Error ! | genre")
        
        ## nation
        try :
            for j in ref_md[i]['nations'] :
                if j['nationNm'] not in nation_list :
                    nation_list.append(j['nationNm'])
                # print(j)
        except :
            print("Error ! | nation")
        
        ## showtype
        for j in ref_md[i]['showTypes'] :
            if j['showTypeGroupNm'] not in showtype_list :
                showtype_list.append(j['showTypeGroupNm'])
            # print(j)
        
        ## type
        try :
            # print(ref_md[i]['typeNm'])
            if ref_md[i]['typeNm'] not in type_list :
                type_list.append(ref_md[i]['typeNm'])
        except :
            print("Error ! | type")
            
        
        ## grade
        try :
            for j in ref_md[i]['audits'] :
                if j['watchGradeNm'] not in grade_list :
                    grade_list.append(j['watchGradeNm'])
                # print(j)
        except :
            print("Error ! | grade")
        
        
    # return pd.DataFrame(result_list, columns = header_list)
    return (actor_list, director_list, genre_list, nation_list, showtype_list, type_list, grade_list)

movie_data_key = construct_movie_key(movie_detail)

19278002버스터 키튼
19278002마리온 맥
19308006게리 쿠퍼
19308006마를렌 디트리히
19328008폴 무니
19358005로버트 도냇
19358005매들린 캐롤
19358007찰스 로튼
19358007클라크 게이블
19358007프랑코트 톤
19368015로버트 테일러
19368015그레타 가르보
19378007폴 로버슨
19378007세드릭 하드윅
19378007로랜드 영
19378023자넷 게이노
19378023프레드릭 마치
19380009캐리 그랜트
19380009캐서린 헵번
19388001에롤 플린
19388001올리비아 드 하빌랜드
19388005진 아서
19388005라이오넬 배리모어
19398001주디 갈란드
19398005헨리 폰다
19398005앨리스 브래디
19398015로렌스 올리비에
19398015메를 오베론
19408005제임스 스튜어트
19408011그리어 가슨
19418007오손 웰즈
19418007도로시 코밍고어
19418007조셉 코튼
19418009바바라 스탠윅
19420001험프리 보가트
19420001잉그리드 버그만
19420001폴 헌레이드
19428001제임스 카그니
19428001조안 레슬리
19428006랜돌프 스콧
19428006존 웨인
19428012캐롤 롬바드
19428012잭 베니
19438005제니퍼 존스
19438005찰스 빅포드
19438006브루스 버네트
19438006J. 캐롤 네이쉬
19438008수잔나 포스터
19438008넬슨 에디
19438010마들레느 솔로뉴
19438010장 마레
19448003빙 크로스비
19448003배리 피츠제럴드
19448005찰스 보이어
19448011레슬리 뱅크스
19448011에스몬드 나이트
19448011로버트 뉴튼
19458003로저 리브시
19458003데보라 카
19458005셀리아 존슨
19458005트레버 하워드
19458008레이 밀랜드
19458008제인 와이먼
19460021유계선
19460021전창근
19460021송억
1

20068576문채원
20068576조재윤
20068576김슬기
20068576이해영
20068576이연두
20068576박민우
20068576이창해
20068576인교진
20068576임영
20068576박태영
20068576정경옥
20068576정라엘
20068576오형근
20068576이범희
20068576이은정
20068576이정은
20068659야마구치 캇페이
20068730클라이브 오웬
20068788파즈 베가
20068788조나 힐
20070008휴 잭맨
20070054조경래
20070054김화수
20070054조명수
20070054홍성택
20070054홍성화
20070054조성래
20070054김희숙
20070054김희윤
20070054박용일
20070054조영래
20070054박지연
20070054박지영
20070054조창휘
20070054변재훈
20070054강성룡
20070054서정화
20070054김대황
20070054김일수
20070054윤병철
20070054장광수
20070054장영진
20070054정미나
20070054박선화
20070054박세영
20070054박순애
20070054박애리
20070054정시영
20070054오희연
20070054정지수
20070054김정복
20070054김순
20070054오경준
20070054김지연
20070068울리히 뮤흐
20070068마르티나 게덱
20070068세바스티안 코치
20070068울리히 터커
20070083벤 위쇼
20070099조경화
20070099김효신
20070099신경호
20070099조영재
20070099서명희
20070099우영제
20070099문현
20070099박근영
20070099오승은
20070099송순섭
20070099온정준
20070099김양휘
20070099이청준
20070114이세영
20070114김윤아
20070114김명환
20070114조용상
20070114김민숙
20070114임지환
20070114박주용
20070115김종수
20070115오만석
20

20112561짐 스터게스
20112589소지섭
20112589한효주
20112589최재훈
20112589김희승
20112589신용철
20112589신은진
20112589최제헌
20112589김현철
20112589최철수
20112589최혜진
20112589심철민
20112589표형렬
20112589안재민
20112589백지영
20112589권지훈
20112589강성대
20112589박갑진
20112589박동규
20112589김성종
20112589인지연
20112589장성희
20112589오정택
20112589최은혜
20112589이길우
20112589윤종화
20112695김효숙
20112695조병찬
20112695조세현
20112695신창수
20112695정한진
20112695강별
20112695박효주
20112695서대현
20112695서왕석
20112695김광섭
20112695차선영
20112695천영암
20112695최명경
20112695윤경호
20112695전승엽
20112695임정환
20112695김신우
20112695이경숙
20112695이유나
20112695성유빈
20112703톰 하디
20112768타이론 파워
20112774톰 이웰
20112775리아넬라 카렐
20112776전영선
20112781정희성
20112781이재희
20112781노형석
20112781박지만
20112781신아가
20112781신준호
20112791이천희
20112791김새론
20112791김아론
20112791정기철
20112791정민우
20112791정성룡
20112791정우상
20112791정재욱
20112791정지민
20112791최가현
20112791최성우
20112791김민수
20112791최영진
20112791이윤경
20112791박경균
20112795정지영
20112795임권택
20112795이진훈
20112795이두용
20112808정기용
20112808승효상
20112808유걸
20112841김영필
20112841신동미
20112841방영배
201128

20123741김기리
20123781봉소악
20123781장효전
20123821정범균
20123821김지민
20124013최고요
20124013이재식
20124013나경찬
20124013박진하
20124013강민아
20124024할리 베리
20124024휴고 위빙
20124031윤다경
20124031심은진
20124031서효명
20124031김동범
20124031정완희
20124031정재필
20124031정희원
20124031김동민
20124031최종렬
20124031박정혁
20124031박형진
20124031송동규
20124034이심결
20124034증국상
20124034장동
20124034장현문
20124034정윤헌
20124034한규석
20124034홍원기
20124039도지한
20124039차인표
20124039조민아
20124039권현상
20124039박정학
20124039임경만
20124039장혜리
20124039조윤경
20124039배진호
20124039안용우
20124039이창용
20124043신이
20124043민아
20124043정애연
20124043경수진
20124043이창대
20124043박준서
20124043김나온
20124044박지빈
20124044김보라
20124044임성호
20124044조정호
20124044이민주
20124044김진태
20124044이주신
20124044박현우
20124044송지선
20124044신동진
20124044우보경
20124044윤상윤
20124047윤시윤
20124047강경덕
20124047박효준
20124047정유신
20124047정진욱
20124047조세영
20124047이민영
20124047김진아
20124050차수연
20124050임남숙
20124050임순옥
20124050장대윤
20124050황남희
20124050박지혜
20124050송성원
20124050오기민
20124050김성태
20124054장서이
20124054한민혁
20124054이승주
20124054최광락
20124054오영석
201

20130703알렉상드르 나혼
20130704찰리 크리드 마일스
20130704레오 그레고리
20130704윌 폴터
20130704리즈 화이트
20130705프랑소아 다미앙
20130705앤느 폴리세비치
20130705세르지 로페즈
20130705잔 해멘넥커
20130706이수정
20130706정은우
20130706신유주
20130706이주엽
20130706주종휘
20130708케이티 창
20130708이스라엘 브로우사드
20130710테렌스 하워드
20130784에밀리 코제트
20130784오렐리 뒤퐁
20130801이타이 슈체르베크
20130801토마슈 솝자크
20130821제임스 프레체빌
20130821벤 멘델슨
20130841슈퍼주니어
20130929강경래
20130929강나연
20130929정광수
20130929최종태
20130929하재민
20130929이아영
20130929김남호
20130929황성우
20130929백승철
20130929조민준
20130952렉스 해리슨
20130952콘스탄스 커밍스
20130961킹스턴 루디스카
20130961성낙원
20130961손형식
20130961조수범
20130961한승진
20130982아우구스티노
20130982타실라
20130982호앙 타인
20130982필립 곤잘레스
20130982와리우라 브후아이야
20130982강가 바스넷
20130982브라욘 쁘쿤톳
20130982캐서린 앤 코르테자
20130982초웁 칸피아루
20130982셰크할 마문
20130982쉬첸밍
20130982셰르조드 아크바로브
20130982오운 라모우
20130982응우옌 반 응우옌
20131001마리아노 마르티네즈
20131001루이사나 로필라토
20131001김민국
20131001송지아
20131003돌프 룬드그렌
20131003스티브 오스틴
20131081줄리아 사라 스톤
20131081마차 그레논
20131081길 벨로우스
20131082조니 루이스
20131082알렉산드라 홀든
20131083케이티 롯츠
20131083캐스

...

20157584탁원정
20157584권문정
20157587김성근
20157587설재훈
20157587이승엽
20157587이승재
20157588콜린 모건
20157591딘 케인
20157591리차드 리엘
20157602브누와 뽀엘부르드
20157603제임스 프리드슨-잭슨
20157603헤이즈 웰포드
20157631장혜민
20157631조혁찬
20157631최승주
20157631노일환
20157781피비 폭스
20157781헬렌 맥크로리
20157782쉐인 하퍼
20157782트리샤 라파체
20157782코리 올리버
20157782하딜 싯투
20157881레베카 퍼거슨
20157883시손 쥰
20157883세키네 츠토무
20157986한고은
20157986배그린
20157986신정선
20157986전현숙
20157986한성아
20157989프랑수아즈 도를리크
20157998롭 브라이든
20158019민복기
20158020패트리샤 닐
20158022박성대
20158042모아 감멜
20158042아니타 월
20158042시몬 J. 베르거
20158042에리크 룬드퀴비스트
20158044미하일 고미아쉬빌리
20158044다치 오르벨라쉬빌리
20158061할룩 빌기너
20158061멜리사 소젠
20158061드멧 아크백
20158061네잣 이슬러
20158068손가람
20158068전예은
20158068조명희
20158068조미연
20158069배진홍
20158082로랑 라피트
20158091박리나
20158091유지원
20158150토비 켑벨
20158161정혜림
20158161안수빈
20158161하강웅
20158161김수빈
20158182김정연
20158182정연
20158182이연서
20158182이원
20158184피터 마크 리치만
20158192마일로 파커
20158192앙케 엥겔케
20158203김영삼
20158203아리
20158203안광연
20158203명지훈
20158240주영
20158241피터 딘클리지
20158241애슐리 벤슨
20158242홍은

20162607케이트 델 카스틸로
20162608김용옥
20162621크리스 J. 머레이
20162621브릿 쇼
20162621아이비 조지
20162629히노 사토시
20162662제동화
20162662이강연
20162662김종인
20162681하나모리 유미리
20162681사쿠라 아야네
20162701임형순
20162721엄국현
20162722배정남
20162722염진환
20162727김가령
20162727미즈키 나나
20162727후루타 아라타
20162741트로이 도나휴
20162741도로시 맥과이어
20162762길예르모 프란셀라
20162762피터 란자니
20162765아이스 큐브
20162765레지나 홀
20162765안소니 앤더슨
20162765이브
20162765숀 패트릭 토마스
20162765가르셀라 보베
20162765우카쉬 암부카
20162765트로이 가리티
20162765디레이 데이비스
20162829제이슨 도링
20162829캐서린 라나사
20162829질리언 머레이
20162829데이빗 오도넬
20162829팀 웨어
20162846조셀린 도나휴
20162846디에고 보네타
20162852윤기원
20162852조용범
20162852최효주
20162869이창섭
20162869이호균
20162869오석규
20162869황민형
20162869김신웅
20162871이준범
20162871정병진
20162871정연서
20162871이상구
20162871홍윤기
20162873로자리오 도슨
20162873마이클 세라
20162875류호연
20162902마크 게티스
20162924유소현
20162924김유연
20162924정찬범
20162925로렌 코핸
20162926마리안느 세이지브레트
20162926CCH 파운더
20162926잭 팰런스
20162928동자건
20162928안탁령
20162928임백굉
20162928오양니니
20162941프란카 포텐테
20162941스털링 제린스
20162962리오넬 메시
20162962호셉 과르디올라
2016296

20170443그웬돌린 크리스티
20170443켈리 마리 트란
20170509이토 카나에
20170513신세휘
20170513양기원
20170513강민준
20170525노넨 레나
20170525호소야 요시마사
20170561채드윅 보스만
20170561다네이 거라이라
20170561레티티아 라이트
20170561다니엘 칼루야
20170561윈스턴 듀크
20170590윤재국
20170590최광일
20170607루이스 맥더겔
20170624사쿠라 유라
20170628아드리안 티티에니
20170656데이빗 하버
20170656가브리엘 유니온
20170656티.아이.
20170656살라 베이커
20170656스티브 콜터
20170656릭 레잇츠
20170669김지환
20170669이진
20170669변지은
20170669송지연
20170669이준성
20170669유은성
20170669차봉주
20170670이혜민
20170670최정인
20170741도미닉 웨스트
20170741바네사 커비
20170741마크 아놀드
20170761로라 프레폰
20170761저스틴 세럭스
20170761리사 쿠드로
20170787한가희
20170841진기주
20170841이광용
20170841김재훈
20170841김주은
20170841오수엽
20170841임정희
20170841김다정
20170841김동혁
20170841최혜림
20170902크래그 힐
20170921샤하브 호세이니
20170921타라네 앨리두스티
20170922이쿠타 토마
20170922키리타니 켄타
20170922미무라
20170922코이케 에이코
20170922에구치 노리코
20170922카키하라 린카
20170931리 반 클리프
20170932프랑코 네로
20170932우디 스토드
20170933에스더 윌리암스
20170941윤성모
20170941조민호
20170941박종훈
20170942문예원
20170942이시형
20170942서영민
20170942양도현
20170942김현창
20170942정원종
20170942김

20176169나카지마 켄토
20176169요시네 쿄코
20176169이시이 안나
20176169칸이치로
20176251권혁범
20176251이호수
20176251김상겸
20176345허윤철
20176421이장미
20176421우진영
20176511막스 리멜트
20176514카메나시 카즈야
20176514나카지마 토모코
20176561줄리아 옌체
20176562마이크 보겔
20176562에리카 크리스틴슨
20176562로버트 포스터
20176581박정섭
20176603짐 벨루시
20176666양민우
20176666진달래
20176717로라 베난티
20176741구민선
20176742크리스토퍼 애봇
20176761후쿠시 소타
20176761쿠도 아스카
20176761요시다 코타로
20176765김남오
20176765박준헌
20176781앤 헤이시
20176784호시노 겐
20176785시모다 쇼타
20176785사이토 소마
20176848재지 비츠
20176917말리 쉘튼
20176961카이라 세드윅
20176962안민상
20176971김기영
20176983이원찬
20176983심승한
20176983사성웅
20177005이예원
20177005이배
20177005배수현
20177005박성범
20177005한경미
20177005김민혜
20177041조배성
20177086멜라니 티에리
20177101김종신
20177104존 시나
20177104미구엘 앙겔 실베스트르
20177144피에르 브라소
20177144세르주 레지아니
20177181클레어 홀트
20177203노성균
20177223김태원
20177223박창수
20177223이영현
20177245여주영
20177245김민
20177245김윤지
20177245최소담
20177245홍가
20177266김다미
20177266윤희정
20177308유관형
20177308장민정
20177308정휘욱
20177308김보민
20177341강병걸
20177341김기용
20177341차진영
20177342양홍석
20177347알렉스

20184529나연
20184529모모
20184529사나
20184529지효
20184529미나
20184529다현
20184529채영
20184529쯔위
20184642장혜정
20184642유인서
20184678안준태
20184678남기주
20184678문득산
20184689김설
20184689이스마일 하지오글루
20184699아녜스 바르다
20184699제이알
20184762사카모토 류이치
20184801알렉스 울프
20184801밀리 샤피로
20184802캐서린 한
20184803이케다 슈이치
20184803우라야마 진
20184803긴가 반죠
20184821후루야 토오루
20184845패트릭 슈왈제네거
20184845퀸 쉐퍼드
20184845니콜라스 쿰베
20184962정재은
20184962양수빈
20184962주민준
20184962김소율
20184962정승원
20184962김준석
20184981고애리
20184981정선철
20184981조혜련
20185001김사라
20185001홍후백
20185046데브라 메싱
20185222빌리 하울
20185222앤-마리 듀프
20185341백규현
20185341김종면
20185341이하영
20185341모그
20185341장민재
20185341진영준
20185341권용석
20185341김영원
20185350박금분
20185350곽두조
20185350강금연
20185350안윤선
20185350박월선
20185350김두선
20185350이원순
20185350박복형
20185350주석희
20185484에카테리나 구세바
20185485라미 말렉
20185485조셉 마젤로
20185485마이크 마이어스
20186121죠 카이리
20186121사사키 미유
20186184크리스티앙 드 라 푸엔테
20186184제시카 세디엘
20186202김지안
20186202장재욱
20186281파멜라 애들론
20186281피터 쿨렌
20186282이와미 마나카
20186284박진용
20186321패트릭 후짓
20186321에단 엠브리
2

In [278]:
actor_table = movie_data_key[0]

md_key_list = ['actor', 'director', 'genre', 'nation', 'showType', 'type', 'grade']
movie_data_table = dict()
for j in range(0,7) :
    cnt = 0
    md_dict = dict()
    for i in movie_data_key[j] :
        md_dict[i] = json.dumps(int(cnt))
        cnt = cnt + 1
    movie_data_table[md_key_list[j]] = md_dict

In [279]:
len(movie_data_table['director'])

4166

In [280]:
len(movie_data_table['actor'])

12381

In [165]:
with open('movie_data_table.json', 'w') as output :
    json.dump(movie_data_table, output)

In [166]:
mdt = dict()

with open('movie_data_table.json', 'r') as f :
    mdt = json.load(f)

movie_data_table['actor']

{'버스터 키튼': '0',
 '마리온 맥': '1',
 '게리 쿠퍼': '2',
 '마를렌 디트리히': '3',
 '폴 무니': '4',
 '로버트 도냇': '5',
 '매들린 캐롤': '6',
 '찰스 로튼': '7',
 '클라크 게이블': '8',
 '프랑코트 톤': '9',
 '로버트 테일러': '10',
 '그레타 가르보': '11',
 '폴 로버슨': '12',
 '세드릭 하드윅': '13',
 '로랜드 영': '14',
 '자넷 게이노': '15',
 '프레드릭 마치': '16',
 '캐리 그랜트': '17',
 '캐서린 헵번': '18',
 '에롤 플린': '19',
 '올리비아 드 하빌랜드': '20',
 '진 아서': '21',
 '라이오넬 배리모어': '22',
 '주디 갈란드': '23',
 '헨리 폰다': '24',
 '앨리스 브래디': '25',
 '로렌스 올리비에': '26',
 '메를 오베론': '27',
 '제임스 스튜어트': '28',
 '그리어 가슨': '29',
 '오손 웰즈': '30',
 '도로시 코밍고어': '31',
 '조셉 코튼': '32',
 '바바라 스탠윅': '33',
 '험프리 보가트': '34',
 '잉그리드 버그만': '35',
 '폴 헌레이드': '36',
 '제임스 카그니': '37',
 '조안 레슬리': '38',
 '랜돌프 스콧': '39',
 '존 웨인': '40',
 '캐롤 롬바드': '41',
 '잭 베니': '42',
 '제니퍼 존스': '43',
 '찰스 빅포드': '44',
 '브루스 버네트': '45',
 'J. 캐롤 네이쉬': '46',
 '수잔나 포스터': '47',
 '넬슨 에디': '48',
 '마들레느 솔로뉴': '49',
 '장 마레': '50',
 '빙 크로스비': '51',
 '배리 피츠제럴드': '52',
 '찰스 보이어': '53',
 '레슬리 뱅크스': '54',
 '에스몬드 나이트': '55',
 '로버트 뉴튼': '56',
 '로저 리브시': '57',
 '데보라

In [43]:
movie_detail['20144444']

{'movieCd': '20144444',
 'movieNm': '아가씨',
 'movieNmEn': 'The Handmaiden',
 'movieNmOg': '',
 'showTm': '144',
 'prdtYear': '2016',
 'openDt': '20160601',
 'prdtStatNm': '개봉',
 'typeNm': '장편',
 'nations': [{'nationNm': '한국'}],
 'genres': [{'genreNm': '스릴러'}, {'genreNm': '드라마'}],
 'directors': [{'peopleNm': '박찬욱', 'peopleNmEn': 'PARK Chan-wook'}],
 'actors': [{'peopleNm': '김민희',
   'peopleNmEn': 'KIM Min-hee',
   'cast': '아가씨',
   'castEn': ''},
  {'peopleNm': '김태리', 'peopleNmEn': 'KIM Tae-ri', 'cast': '하녀', 'castEn': ''},
  {'peopleNm': '하정우', 'peopleNmEn': 'HA Jung-woo', 'cast': '백작', 'castEn': ''},
  {'peopleNm': '조진웅',
   'peopleNmEn': 'CHO Jin-woong',
   'cast': '이모부',
   'castEn': ''},
  {'peopleNm': '김해숙',
   'peopleNmEn': 'KIM Hae-sook',
   'cast': '여집사',
   'castEn': ''},
  {'peopleNm': '문소리', 'peopleNmEn': 'MOON So-ri', 'cast': '이모', 'castEn': ''},
  {'peopleNm': '최병모',
   'peopleNmEn': 'CHOE Byeong-mo',
   'cast': '손님',
   'castEn': ''},
  {'peopleNm': '김인우', 'peopleNmEn': 'K

### movie.csv

In [243]:
## movie.csv
# final dataset
# movieCd = index
# salesAcc(target)
# scrnAcc(target)
# audiAcc(target)
# showAcc(target)
# showTm
# rating
# type
# genres(multi)
#   genreNm
# directors(multi)
#   directorNm
# actors(multi)
#   actorNm
# showTypes(multi)
#   showTypeNm
# companys(multi)
#   companyCd
# watchGrade
# nations(multi)
#   nationNm

# with open('movie_detail.json', 'r') as f :
#     movie_detail = json.load(f)
# 
#
movie_rating_list = pd.read_csv('movie_rating_list.csv')


movie_csv_header = ['movieCd',       # 0
                    'showTm',        # 1
                    'rating',        # 2
                    'type',          # 3
                    'genre_1',       # 4
                    'genre_2',       # 5
                    'genre_3',       # 6
                    'director_1',    # 7
                    'director_2',    # 8
                    'actor_1',       # 9
                    'actor_2',       # 10
                    'actor_3',       # 11
                    'actor_4',       # 12
                    'actor_5',       # 13
                    'showType_1',    # 14
                    'showType_2',    # 15
                    'showType_3',    # 16
                    'company_1',     # 17
                    'company_2',     # 18
                    'company_3',     # 19
                    'company_4',     # 20
                    'grade',         # 21
                    'nation_1',      # 22
                    'nation_2',      # 23
                    'multi',         # 24
                    'domestic',      # 25
                    'salesAcc',      # 26
                    'scrnAcc',       # 27
                    'audiAcc',       # 28
                    'showAcc'        # 29
                   ]

movie_result = []

for i in movie_detail :
    md = movie_detail[i]
    tmp_list = []
    
    tmp_list.append(md['movieCd'])
    
    tmp_list.append(md['showTm'])
    
    k = movie_rating_list[movie_rating_list['movieCd'] == md['movieCd']]
    try :
        tmp_list.append(k.iloc[0].rating)
    except :
        print(i)
    
    tmp_list.append(movie_data_table['type'][md['typeNm']])
    
    cnt = 0
    for j in md['genres'] :
        tmp_list.append(movie_data_table['genre'][j['genreNm']])
        cnt = cnt + 1
        if cnt == 3 :
            break
    for j in range(cnt,3) :
        tmp_list.append(float('nan'))
    
    cnt = 0
    for j in md['directors'] :
        tmp_list.append(movie_data_table['director'][j['peopleNm']])
        cnt = cnt + 1
        if cnt == 2 :
            break
    for j in range(cnt, 2) :
        tmp_list.append(float('nan'))
        
    cnt = 0
    for j in md['actors'] :
        tmp_list.append(movie_data_table['actor'][j['peopleNm']])
        cnt = cnt + 1
        if cnt == 5 :
            break
    for j in range(cnt, 5) :
        tmp_list.append(float('nan'))
    
    cnt = 0
    for j in md['showTypes'] :
        tmp_list.append(movie_data_table['showType'][j['showTypeGroupNm']])
        cnt = cnt + 1
        if cnt == 3 :
            break
    for j in range(cnt, 3) :
        tmp_list.append(float('nan'))
    
    cnt = 0
    for j in md['companys'] :
        tmp_list.append(j['companyCd'])
        cnt = cnt + 1
        if cnt == 4 :
            break
    for j in range(cnt, 4) :
        tmp_list.append(float('nan'))
        
    try :
        tmp_list.append(movie_data_table['grade'][md['audits'][0]['watchGradeNm']])
    except :
        tmp_list.append(float('nan'))
        
    cnt = 0
    for j in md['nations'] :
        tmp_list.append(movie_data_table['nation'][j['nationNm']])
        cnt = cnt + 1
        if cnt == 2 :
            break
    for j in range(cnt, 2) :
        tmp_list.append(float('nan'))
    
    try :
        k = movie_data[movie_data['movieCd'] == md['movieCd']]
        k = k.iloc[0]
        # print(k)
        if(k['multi'] == 'Y') :
            tmp_list.append(1)
        else :
            tmp_list.append(0)
            
        if(k['nation'] == 'F') :
            tmp_list.append(0)
        else :
            tmp_list.append(1)

        tmp_list.append(k['salesAcc'])

        #screen
        tmp_list.append(k['screen'])

        #audiAcc
        tmp_list.append(k['audienceAcc'])

        #show
        tmp_list.append(k['show'])
    except :
        print('..')
    # print(tmp_list)
    movie_result.append(tmp_list)
    

movie = pd.DataFrame(movie_result, columns = movie_csv_header)
movie.to_csv('movie.csv')

#tmp_list.to_csv(tmp_list, movie_csv_header)

20122066
..
20124356
..
20136450
..
